In [17]:
import pickle

# Replace 'trained_agents_data.pkl' with the path to your .pkl file
file_path = 'marketing_strategy_data.pkl'

# Open the file in binary read mode
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Now 'data' contains the deserialized object from the .pkl file
print(data)

{'Lead Market Analyst': {'suggestions': ['Focus on improving the structure and clarity of the initial output.', 'Incorporate more detailed competitor information, including funding and market focus for improved context.', 'Enhance audience engagement strategies with actionable tactics and specific channels for engagement.', 'Ensure the conclusion synthesizes the insights clearly to provide a strong closing message.'], 'quality': 9.0, 'final_summary': '1. Rewrite to improve clarity and organization. 2. Expand on competitor analysis with funding details. 3. Include specific marketing strategies tailored for target demographics. 4. Conclude with a strong synthesis of the findings.'}, 'Chief Marketing Strategist': {'suggestions': ["Include specific use cases in the marketing strategy to better illustrate the application of CrewAI's solutions.", 'Ensure all key performance indicators (KPIs) are measurable and specific to track effectiveness accurately.', 'Reiterate the importance of the tar

In [ ]:
from typing import List
from crewai import Agent, Crew, Process, Task, LLM
from textwrap import dedent
import os
from dotenv import load_dotenv
load_dotenv()

# Check our tools documentations for more information on how to use them
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from pydantic import BaseModel, Field

class MarketStrategy(BaseModel):
	"""Market strategy model"""
	name: str = Field(..., description="Name of the market strategy")
	tatics: List[str] = Field(..., description="List of tactics to be used in the market strategy")
	channels: List[str] = Field(..., description="List of channels to be used in the market strategy")
	KPIs: List[str] = Field(..., description="List of KPIs to be used in the market strategy")

class CampaignIdea(BaseModel):
	"""Campaign idea model"""
	name: str = Field(..., description="Name of the campaign idea")
	description: str = Field(..., description="Description of the campaign idea")
	audience: str = Field(..., description="Audience of the campaign idea")
	channel: str = Field(..., description="Channel of the campaign idea")

class Copy(BaseModel):
	"""Copy model"""
	title: str = Field(..., description="Title of the copy")
	body: str = Field(..., description="Body of the copy")

class MarketingPostsCrew():
	"""MarketingPosts crew"""

	def __init__(self):
		self.llm = LLM(
			api_key = os.getenv("AZURE_API_KEY"),
			api_version = os.getenv("AZURE_API_VERSION"),
			base_url = os.getenv("AZURE_API_BASE"),
			model = 'azure/gpt-4o-mini',
			azure=True
		)		
	
	def lead_market_analyst(self) -> Agent:
		return Agent(
			role='Lead Market Analyst',
			goal='Conduct amazing analysis of the products and competitors, providing in-depth insights to guide marketing strategies.',
			backstory='As the Lead Market Analyst at a premier digital marketing firm, you specialize in dissecting online business landscapes.',
			tools=[SerperDevTool(), ScrapeWebsiteTool()],
			llm=self.llm,
			function_calling_llm=self.llm,
			verbose=True,
			memory=False,
		)

	def chief_marketing_strategist(self) -> Agent:
		return Agent(
            role='Chief Marketing Strategist',
            goal='Synthesize amazing insights from product analysis to formulate incredible marketing strategies.',
            backstory='You are the Chief Marketing Strategist at a leading digital marketing agency, known for crafting bespoke strategies that drive success.',
			tools=[SerperDevTool(), ScrapeWebsiteTool()],
			llm=self.llm,
			function_calling_llm=self.llm,
			verbose=True,
			memory=False,
		)

	def creative_content_creator(self) -> Agent:
		return Agent(
            role='Creative Content Creator',
            goal='Develop compelling and innovative content for social media campaigns, with a focus on creating high-impact ad copies.',
            backstory='As a Creative Content Creator at a top-tier digital marketing agency, you excel in crafting narratives that resonate with audiences. Your expertise lies in turning marketing strategies into engaging stories and visual content that capture attention and inspire action.',
			llm=self.llm,
			function_calling_llm=self.llm,
			verbose=True,
			memory=False,
		)

	def research_task(self) -> Task:
		return Task(
            description=dedent("""
                Conduct a thorough research about the customer and competitors in the context
                of {customer_domain}.
                Make sure you find any interesting and relevant information given the
                current year is 2024.
                We are working with them on the following project: {project_description}."""),
            expected_output=dedent("""
                A complete report on the customer and their customers and competitors,
                including their demographics, preferences, market positioning and audience engagement."""),			
			agent=self.lead_market_analyst()
		)

	def project_understanding_task(self) -> Task:
		return Task(
			description=dedent("""
                Understand the project details and the target audience for
                {project_description}.
                Review any provided materials and gather additional information as needed."""),
            expected_output="A detailed summary of the project and a profile of the target audience.",
			agent=self.chief_marketing_strategist()
		)

	def marketing_strategy_task(self) -> Task:
		return Task(
			description=dedent("""
                Formulate a comprehensive marketing strategy for the project
                {project_description} of the customer {customer_domain}.
                Use the insights from the research task and the project understanding
                task to create a high-quality strategy."""),
            expected_output=dedent("""
                A detailed marketing strategy document that outlines the goals, target
                audience, key messages, and proposed tactics, make sure to have name, tatics, channels and KPIs"""),
			agent=self.chief_marketing_strategist(),
			output_json=MarketStrategy
		)

	def campaign_idea_task(self) -> Task:
		return Task(
            description=dedent("""
                Develop creative marketing campaign ideas for {project_description}.
                Ensure the ideas are innovative, engaging, and aligned with the overall marketing strategy."""),
            expected_output="A list of 5 campaign ideas, each with a brief description and expected impact.",
			agent=self.creative_content_creator(),
   		    output_json=CampaignIdea
		)

	def copy_creation_task(self) -> Task:
		return Task(
			description=dedent("""
                Create marketing copies based on the approved campaign ideas for {project_description}.
                Ensure the copies are compelling, clear, and tailored to the target audience."""),
            expected_output="Marketing copies for each campaign idea.",
			agent=self.creative_content_creator(),
   		    context=[self.marketing_strategy_task(), self.campaign_idea_task()],
			output_json=Copy
		)

	def crew(self) -> Crew:
		"""Creates the MarketingPosts crew"""
		return Crew(
			agents=[self.lead_market_analyst(), self.chief_marketing_strategist(), self.creative_content_creator()],
			tasks=[self.research_task(), self.project_understanding_task(), self.marketing_strategy_task(), self.campaign_idea_task(), self.copy_creation_task()], 
			process=Process.sequential,
			function_calling_llm=self.llm,
			verbose=True,
			# process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
		)


In [15]:
def run():
    # Replace with your inputs, it will automatically interpolate any tasks and agents information
    inputs = {
        'customer_domain': 'crewai.com',
        'project_description': """
CrewAI, a leading provider of multi-agent systems, aims to revolutionize marketing automation for its enterprise clients. This project involves developing an innovative marketing strategy to showcase CrewAI's advanced AI-driven solutions, emphasizing ease of use, scalability, and integration capabilities. The campaign will target tech-savvy decision-makers in medium to large enterprises, highlighting success stories and the transformative potential of CrewAI's platform.

Customer Domain: AI and Automation Solutions
Project Overview: Creating a comprehensive marketing campaign to boost awareness and adoption of CrewAI's services among enterprise clients.
"""
    }
    return MarketingPostsCrew().crew().kickoff(inputs=inputs)


def train():
    """
    Train the crew for a given number of iterations.
    """
    inputs = {
        'customer_domain': 'crewai.com',
        'project_description': """
CrewAI, a leading provider of multi-agent systems, aims to revolutionize marketing automation for its enterprise clients. This project involves developing an innovative marketing strategy to showcase CrewAI's advanced AI-driven solutions, emphasizing ease of use, scalability, and integration capabilities. The campaign will target tech-savvy decision-makers in medium to large enterprises, highlighting success stories and the transformative potential of CrewAI's platform.

Customer Domain: AI and Automation Solutions
Project Overview: Creating a comprehensive marketing campaign to boost awareness and adoption of CrewAI's services among enterprise clients.
"""
    }
    try:
        MarketingPostsCrew().crew().train(n_iterations=2, filename="marketing_strategy_data.pkl", inputs=inputs)

    except Exception as e:
        raise Exception(f"An error occurred while training the crew: {e}")

In [10]:
output = run()

2024-11-09 14:02:33,011 - 140241925185664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Market Analyst
## Task: 
Conduct a thorough research about the customer and competitors in the context
of crewai.com.
Make sure you find any interesting and relevant information given the
current year is 2024.
We are working with them on the following project: 
CrewAI, a leading provider of multi-agent systems, aims to revolutionize marketing automation for its enterprise clients. This project involves developing an innovative marketing strategy to showcase CrewAI's advanced AI-driven solutions, emphasizing ease of use, scalability, and integration capabilities. The campaign will target tech-savvy decision-makers in medium to large enterprises, highlighting success stories and the transformative potential of CrewAI's platform.

Customer Domain: AI and Automation Solutions
Project Overview: Creating a comprehensive marketing campaign to boost awareness and adoption of CrewAI's services among enterprise clients.
.


# Agent: Lead Market Analyst
## Thought: I need to gather 

In [11]:
from IPython.display import Markdown, display
display(Markdown(output.raw))

**Campaign Idea 1: “Empower Your Marketing with AI”**  
*Ad Copy 1: “Transform Your Marketing Game! 🚀 Discover how CrewAI’s cutting-edge multi-agent systems simplify marketing automation. Drive efficiency, enhance decision-making, and elevate customer engagement—all in one scalable platform. Join the AI revolution today!”*

*Ad Copy 2: “Want to work smarter, not harder? With CrewAI, integrate advanced AI solutions that effortlessly adapt to your business needs. Say goodbye to mundane tasks and hello to powerful automation. Experience unparalleled growth—let’s get started!”*

---

**Campaign Idea 2: “Success Stories”**  
*Ad Copy 1: “Real Results, Real Transformation! 🌟 See how leading enterprises have harnessed CrewAI’s innovative platform to boost productivity and achieve marketing excellence. Don’t just take our word for it—discover the success stories that are shaping the future of automation!”*

*Ad Copy 2: “Is your marketing strategy ready for an upgrade? Explore how our clients have redefined their marketing efforts with CrewAI’s AI-driven solutions. Join the ranks of transformation—because success is just a click away!”*

---

**Campaign Idea 3: “Seamless Integration”**  
*Ad Copy 1: “Integrate with Ease! 🔗 Experience the power of seamless integration with CrewAI—no technical expertise needed! Our solutions fit right into your existing systems, enhancing your marketing capabilities without disruption. Efficiency has never been this simple!”*

*Ad Copy 2: “Ready to enhance your tech stack? CrewAI’s AI-driven platform seamlessly integrates with your existing tools, providing you with a holistic marketing solution. Upgrade your approach, maintain your workflow, and maximize your potential. Let’s connect!”*

---

**Campaign Idea 4: “Scalability”**  
*Ad Copy 1: “Scale with Confidence! 📈 No matter the size of your enterprise, CrewAI empowers you to grow effortlessly. From startups to Fortune 500 companies, our scalable solutions cater to your evolving needs. Discover how we can fuel your next growth phase!”*

*Ad Copy 2: “Is your marketing strategy scalable? With CrewAI, growth is a given. Our multi-agent systems allow you to adapt and expand without losing momentum. Unlock new possibilities with a partner that understands your ambition!”*

---

**Campaign Idea 5: “Innovation in Marketing Automation”**  
*Ad Copy 1: “Innovate or Get Left Behind! 🌍 CrewAI is at the forefront of marketing automation. Our intelligent solutions combine advanced AI technologies with user-friendly interfaces to give you the edge you need in today’s competitive landscape. Embrace innovation with us!”*

*Ad Copy 2: “Take the leap into the future of marketing! With CrewAI’s innovative systems, you can automate intelligently, strategize effectively, and engage more meaningfully. Don’t let your competition outpace you—become a market leader today!”*

---

These compelling and clear marketing copies are tailored to tech-savvy decision-makers in medium to large enterprises, showcasing CrewAI’s unique value propositions and the transformative potential of its solutions. The outcome is a cohesive and engaging campaign that is set to boost awareness and adoption of CrewAI's services.

In [16]:
train()

2024-11-09 14:11:01,200 - 140241925185664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
2024-11-09 14:11:01,230 - 140241925185664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Market Analyst
## Task: 
Conduct a thorough research about the customer and competitors in the context
of crewai.com.
Make sure you find any interesting and relevant information given the
current year is 2024.
We are working with them on the following project: 
CrewAI, a leading provider of multi-agent systems, aims to revolutionize marketing automation for its enterprise clients. This project involves developing an innovative marketing strategy to showcase CrewAI's advanced AI-driven solutions, emphasizing ease of use, scalability, and integration capabilities. The campaign will target tech-savvy decision-makers in medium to large enterprises, highlighting success stories and the transformative potential of CrewAI's platform.

Customer Domain: AI and Automation Solutions
Project Overview: Creating a comprehensive marketing campaign to boost awareness and adoption of CrewAI's services among enterprise clients.
.


# Agent: Lead Market Analyst
## Thought: I need to gather 